# Python script for data transformation 

## BRICARE:

BRICARE consists of 2 different types of files by year:

a. File after 2022 (2023-2024) = 79 kolom


b. File before 2022 (2019-2022) = 27 kolom


### File Type A


Data Extraction for File Type A must be 2 Files:


A.1 Columns (without "Details")


A.2 Details only 

Columns to be cleansed or Transform:
- All columns with values "None", "NaN, "N/A", "NULL"
- These columns must follow this datetime format: format='%Y-%m-%d %H:%M:%S' or format='%Y-%m-%d %H:%M:%S.%f' 

['Create_Date','TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']

- Remove all unknown characters e.g. \ufeff in column "Ticket_ID"

- Columns shoud be mapped based on their Call_Type_ID:

['Produk','Jenis_Produk','Jenis_Laporan']

- PLEASE ADD CIF

#### File A.1 Columns (without "Details"). Please use this if the file is txt

In [7]:
import pandas as pd
import re
import numpy as np

# 78 Columns
column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

def parse_file(file_path):

    data = []
    date_pattern = re.compile(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}')

    with open(file_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            parts = line.strip().split(';')

            date_index = next(i for i, part in enumerate(parts) if date_pattern.match(part))

            ticket_id = parts[0] 
            call_type_id = parts[1]  
            description = ';'.join(parts[2:date_index])  
            create_date = parts[date_index]  

      
            data.append([ticket_id, call_type_id, description, create_date] + parts[date_index + 1:])


    df = pd.DataFrame(data, columns=column_names)

    df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')

    return df


file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_1masking.txt"

df = parse_file(file_path)
df.replace('NULL', np.nan, inplace=True)
df.replace('None', np.nan, inplace=True)
df.replace('N/A', np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')


columns_to_convert = ['TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']
for column in columns_to_convert:
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S', errors='coerce')

    df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)
   

df['Ticket_ID'] = df['Ticket_ID'].apply(lambda x: x.replace('\ufeff', '').strip())



df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')
startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_uat{formatted_startdate}_{formatted_enddate}.csv"


df.to_csv(filename, index=False)

print(f"Data saved to {filename}")

Data saved to bricare_uat20230101_20230101.csv


C:\Users\maste\AppData\Local\Temp\ipykernel_65008\3996884930.py:51: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('NULL', np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_65008\3996884930.py:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('None', np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_65008\3996884930.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with

#### File A.1 Columns (without "Details"). Please use this if the file is csv

In [46]:
import pandas as pd
import numpy as np

# 78 Columns
column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

def parse_csv(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            parts = line.strip().split(';')
            if len(parts) > 78:
                description = ';'.join(parts[2:-75])
                new_parts = parts[:2] + [description] + parts[-75:]
                data.append(new_parts)
            else:
                data.append(parts)
    
    df = pd.DataFrame(data, columns=column_names)
    df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')
    
    df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
    df.fillna('', inplace=True)
    df = df.replace(['0', 0], '')

    columns_to_convert = ['TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']
    for column in columns_to_convert:
        df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S', errors='coerce')
        df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)
    
    df['Ticket_ID'] = df['Ticket_ID'].apply(lambda x: x.replace('\ufeff', '').strip())

    return df
file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_1masking.csv"
df = parse_csv(file_path)

startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_{formatted_startdate}_{formatted_enddate}.csv"
df.to_csv(filename, index=False)

filename


C:\Users\maste\AppData\Local\Temp\ipykernel_51572\2385745217.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_51572\2385745217.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


'bricare_20230101_20230101.csv'

## Cleasing the master Call Type file

In [34]:
import pandas as pd

master_df_path = r"C:\Users\maste\Downloads\bricare\(REVISED) SLA-OLA_NewUserGrouping_Ringkasan Kirim ME Versi 1.6.csv"
df = pd.read_csv(master_df_path, sep=';')


df.replace('NULL', np.nan, inplace=True)
df.replace('None', np.nan, inplace=True)
df.replace('N/A', np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')
df = df.dropna(how='all')
df.iloc[:450]
df.to_csv("master_calltype.csv", index=False)

C:\Users\maste\AppData\Local\Temp\ipykernel_24036\2288781540.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


## Call Type di tes MMS

In [2]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\Data Mapping - Case_Type.csv"
df=pd.read_csv(path)
# case_types = ['8634', '8635', '8636', '8623', '8624', '8628', '8629', '8630', '8412', '8625', '8615', '8616', '8617', '8618', '8619', '8638', '8620']

case_types = ['8412']
# Filter the dataframe
df = df[df['Case Types'].isin(case_types)]
df['Segment'] = df['Segment'].replace('All', '')
df
df.to_csv(r'C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\Case_Type_MMS2.csv', index=False)

## Call type final add external ID

In [17]:
import pandas as pd


file_path = r"D:\dataquality2\new as per 5 June\error060824083047752_SIT.csv"
data = pd.read_csv(file_path)

df.loc[df['Case Types'].isin([8418, 8419]), 'PRODUCT'] = 'Transaction Banking'
df.loc[df['Case Types'].isin([8708, 8711]), 'PRODUCT'] = 'Transaction Banking'
df['CASE CATEGORY'] = df['CASE CATEGORY'].replace('Customer Feedback / Sentiment Monitoring', 'Customer Feedback')

segment_mapping = {
    'Individu': 'Individu Umum',
    'Individu Umum, Individu Priority &amp; Private' : 'Individu Umum',
    'All' : 'Individu Umum',
    'Individu Umum, UMKM' : 'Individu Umum',
    
}

product_mapping = {
    'loans': 'Loans',
    'savings': 'Savings',
    'KTA': ''
}

subproduk_mapping = {
    'Giro': 'Giro (CA)',
    'KPR, KUR, KMK, KI': '',
    'SBN': '',
    'DPLK': ''
}

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
data["ERROR"]

0     Customer Segment: bad value for restricted picklist field: Individu Umum, Individu Priority &amp; Private
1     Customer Segment: bad value for restricted picklist field: Individu Umum, Individu Priority &amp; Private
2     Customer Segment: bad value for restricted picklist field: Individu Umum, Individu Priority &amp; Private
3     Customer Segment: bad value for restricted picklist field: Individu Umum, Individu Priority &amp; Private
4                                                                 bad value for restricted picklist field: Giro
5                                                Customer Segment: bad value for restricted picklist field: All
6                                                Customer Segment: bad value for restricted picklist field: All
7                                                         Product: bad value for restricted picklist field: SBN
8                                                        Product: bad value for restricted picklist fiel

In [12]:
import pandas as pd

path=r"D:\dataquality2\new as per 5 June\error060824061742698_uatcalltype.csv"
df=pd.read_csv(path)
df=df.iloc[:50]
df['SEGMENT']='Individu Umum'
df.to_csv('error060824061742698_uatcalltype_resolved.csv', index=False)


### See the error in Segment Column

In [41]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\Data Mapping - Case_Type.csv"
df = pd.read_csv(path)

# df['Product'].unique()

# I have a dataset, I want in column Segment you map
# Individu=Individu umum
# All becomes empty
# and column Product

# loans = Loans
# savings = Savings



# mapping

segment_mapping = {
    'Individu': 'Individu umum',
}

product_mapping = {
    'loans': 'Loans',
    'savings': 'Savings',
    'KTA' : ''
}

subproduk_mapping = {
    'Giro' : 'Giro (CA)',
    'KPR, KUR, KMK, KI' : '',
    'SBN' : '',
    'DPLK' : ''
}

case_category = {
    'Customer Feedback / Sentiment Monitoring' : 'Customer Feedback'
}


df['Segment'] = df['Segment'].apply(lambda x: segment_mapping.get(x, ''))
df['Product'] = df['Product'].map(product_mapping).fillna(df['Product'])

df.fillna('', inplace=True)

print(df['Segment'].unique())

# df['Product'].unique()

# output=r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\master_casetype_cleaned.csv"

# See my code above I wnat another mapping in column Case Category and Sub Produk

df.to_csv(output, index=False)

['' 'Individu umum']


C:\Users\maste\AppData\Local\Temp\ipykernel_40000\1757643033.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


In [53]:
import pandas as pd

# path= r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error060424015731096.csv"
# path2 = r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error060424020750586.csv"
path3 = r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error060424022013225.csv"
df = pd.read_csv(path3)

# segment_mapping = {
#     'Individu': 'Individu umum',
# }

# product_mapping = {
#     'loans': 'Loans',
#     'savings': 'Savings',
#     'KTA': ''
# }

# subproduk_mapping = {
#     'Giro': 'Giro (CA)',
#     'KPR, KUR, KMK, KI': '',
#     'SBN': '',
#     'DPLK': ''
# }

# case_category = {
#     'Customer Feedback / Sentiment Monitoring': 'Customer Feedback',
#     # Add more mappings for other Case Category values here
# }

# # Apply mapping to columns
# df['SEGMENT'] = df['SEGMENT'].apply(lambda x: segment_mapping.get(x, ''))
# df['PRODUCT'] = df['PRODUCT'].map(product_mapping).fillna(df['PRODUCT'])
# df['CASE CATEGORY'] = df['CASE CATEGORY'].apply(lambda x: case_category.get(x, x))  # Apply default value if not mapped
# df['SUB PRODUCT'] = df['SUB PRODUCT'].map(subproduk_mapping).fillna(df['SUB PRODUCT'])
df.fillna('', inplace=True)
df.replace('Transaction Solution', '', inplace=True)
output=r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\master_casetype_cleaned_error3.csv"

# df['SUB PRODUK'] = df.apply(lambda row: row['SUB PRODUK'].replace('Transaction Solution', '') if 'Servicing' in row['PRODUK'] else row['SUB PRODUK'], axis=1)
# Copy values from NEW CASE DESCRIPTION to CASE DESCRIPTION if empty
df['CASE DESCRIPTION']='Nasabah Mengajukan Unlock User Microsite DPLK'
df

df.to_csv(output, index=False)

C:\Users\maste\AppData\Local\Temp\ipykernel_40000\3962062038.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


#### Call type mapping for columns 'Produk', 'Jenis Produk', 'Jenis Laporan'

In [8]:
import pandas as pd


user_dataset_path = r"D:\dataquality2\bricare_uat20230101_20230101.csv"
user_df = pd.read_csv(user_dataset_path)
master_df_path = r"D:\dataquality\master_calltype.csv"
master_df = pd.read_csv(master_df_path)


master_df = master_df.rename(columns={
    'Case Types': 'Call_Type_ID', 
    'Product': 'Produk', 
    'Sub Product': 'Jenis_Produk', 
    'Case Category': 'Jenis_Laporan'
})


user_df['Call_Type_ID'] = user_df['Call_Type_ID'].astype(str)
master_df['Call_Type_ID'] = master_df['Call_Type_ID'].astype(str)


merged_df = pd.merge(user_df, master_df[['Call_Type_ID', 'Produk', 'Jenis_Produk', 'Jenis_Laporan']], on='Call_Type_ID', how='left')

user_df['Produk'] = merged_df['Produk_y']
user_df['Jenis_Produk'] = merged_df['Jenis_Produk_y']
user_df['Jenis_Laporan'] = merged_df['Jenis_Laporan_y']



user_df.to_csv(user_dataset_path, index=False)

#### File A.2 Details only. Please use this if the file is txt

In [9]:
import pandas as pd

def process_text_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove BOM from each line
    lines = [line.replace('\ufeff', '') for line in lines]

    entries = []
    current_entry = []
    current_ticket_id = None

    for line in lines:
        if line.startswith('TTB'):
            if current_entry:  
                entries.append((current_ticket_id, '\n'.join(current_entry)))
                current_entry = []
        
            parts = line.split(',', 3)
            if len(parts) > 3:
                current_ticket_id = parts[0]  
                current_entry.append(parts[3].strip())  
            continue
        current_entry.append(line.strip())

    if current_entry:
        entries.append((current_ticket_id, '\n'.join(current_entry)))

    return entries


def remove_bom_and_strip(df):
    return df.applymap(lambda x: x.replace('\ufeff', '').strip() if isinstance(x, str) else x)


file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_2_details.txt"
processed_data = process_text_data(file_path)


df_final = pd.DataFrame(processed_data, columns=['Ticket ID', 'Details'])

if df_final.iloc[0]['Ticket ID'] and df_final.iloc[0]['Details'].startswith(df_final.iloc[0]['Ticket ID']):
    df_final.at[0, 'Details'] = df_final.iloc[0]['Details'][len(df_final.iloc[0]['Ticket ID'])+2:]

# df_final=df_final.iloc[:10]
df_final.iloc[:10].to_csv('details_uat_20230101_20230101.csv', index=False)



#### File A.2 Details only. Please use this if the file is csv

In [49]:
import pandas as pd

def process_csv_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove BOM from each line
    lines = [line.replace('\ufeff', '') for line in lines]

    entries = []
    current_entry = []
    current_ticket_id = None

    for line in lines:
        if line.startswith('TTB'):
            if current_entry:
                entries.append((current_ticket_id, '\n'.join(current_entry)))
                current_entry = []
        
            parts = line.split(',', 3)
            if len(parts) > 3:
                current_ticket_id = parts[0]
                current_entry.append(parts[3].strip())
            continue
        current_entry.append(line.strip())

    if current_entry:
        entries.append((current_ticket_id, '\n'.join(current_entry)))

    return entries

file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_2_details.csv"
processed_data = process_csv_data(file_path)

df_final = pd.DataFrame(processed_data, columns=['Ticket ID', 'Details'])


if df_final.iloc[0]['Ticket ID'] and df_final.iloc[0]['Details'].startswith(df_final.iloc[0]['Ticket ID']):
    df_final.at[0, 'Details'] = df_final.iloc[0]['Details'][len(df_final.iloc[0]['Ticket ID'])+2:]


df_final = df_final.iloc[:10]
output_path = "details_20230101_20230101.csv"
df_final.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")


Processed data saved to details_20230101_20230101.csv


#### Merge the file A.1 and file A.2

In [50]:
import pandas as pd

#just take 10 lines for an example
path=r"D:\dataquality\bricare_20230101_20230101.csv"
df=pd.read_csv(path)
df.iloc[:10].to_csv(path,index=False)

In [12]:
import pandas as pd


file_path_1 = r"D:\dataquality2\bricare_uat20230101_20230101.csv"
file_path_2 = r"D:\dataquality2\details_uat_20230101_20230101.csv"


df_tenline_bricare = pd.read_csv(file_path_1)
df_detail_bricare_10line = pd.read_csv(file_path_2)

df_detail_bricare_10line.columns = ['Ticket_ID', 'Details']

merged_df = pd.merge(df_tenline_bricare, df_detail_bricare_10line, on='Ticket_ID', how='left')


output_file_path = r"D:\dataquality2\bricare_uat_20230101_20230101.csv"


column_to_move="Details"
merged_df = merged_df[[col for col in merged_df if col != column_to_move][:3] + [column_to_move] + [col for col in merged_df if col != column_to_move][3:]] 

merged_df.to_csv(output_file_path, index=False)

### File Type B


Data Extraction for File Type B (27 columns)


Columns to be cleansed or Transform:
- All columns with values "None", "NaN, "N/A", "NULL"
- These columns must follow this datetime format: format='%Y-%m-%d %H:%M:%S' or format='%Y-%m-%d %H:%M:%S.%f' 

['TanggalClosed', 'tanggalTransaksi','Create_Date']

- Remove all unknown characters e.g. \ufeff in column "Ticket_ID" if any

- PLEASE ADD CIF



In [52]:
import pandas as pd
import numpy as np

# Define the column list
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"  
]


path = r"C:\Users\maste\Downloads\BRICARE_25042024 masking.csv"


data = pd.read_csv(path, delimiter=';')


data['Column1'] = data['Column1'].astype(str)
data_cleaned = data[data['Column1'].str.match(r'TTB\d+')]

data_cleaned['Column2'] = data_cleaned['Column2'].astype(str)
data_cleaned = data_cleaned[data_cleaned['Column2'].str.match(r'^\d{4}$')]

data_cleaned['Column4'] = pd.to_datetime(data_cleaned['Column4'], errors='coerce')
data_cleaned = data_cleaned.dropna(subset=['Column4'])


data_to_drop = ['Column28', 'Column29', 'Column30', 'Column31', 'Column32']
data_cleaned = data_cleaned.drop(columns=data_to_drop)


if len(data_cleaned.columns) <= len(column_list):
    data_cleaned.columns = column_list[:len(data_cleaned.columns)]


data_cleaned.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
data_cleaned.fillna('', inplace=True)
data_cleaned = data_cleaned.replace(['0', 0], '')


columns_to_convert = ['TanggalClosed', 'tanggalTransaksi', 'Create_Date']
for column in columns_to_convert:
    data_cleaned[column] = pd.to_datetime(data_cleaned[column], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
    data_cleaned[column] = data_cleaned[column].apply(lambda x: '' if pd.isna(x) else x)

# Just take 10 lines for an example
data_cleaned = data_cleaned.iloc[:10]


data_cleaned.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')


startdate = pd.Timestamp(min(data_cleaned['Create_Date']))
enddate = pd.Timestamp(max(data_cleaned['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_{formatted_startdate}_{formatted_enddate}.csv"


data_cleaned.to_csv(filename, index=False)

print(f"Data saved to {filename}")


C:\Users\maste\AppData\Local\Temp\ipykernel_33500\3452762791.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['Column2'] = data_cleaned['Column2'].astype(str)


Data saved to bricare_20200101_20200101.csv


## Check the number of columns

In [55]:

path1=r"D:\dataquality\bricare_20230101_20230101.csv"
path2=r"D:\dataquality\bricare_20200101_20200101.csv"

df1=pd.read_csv(path1)
df2=pd.read_csv(path2)
print(len(df1.columns))
print(len(df2.columns))

print(df1.columns)
print(df2.columns)

79
27
Index(['Ticket_ID', 'Call_Type_ID', 'Call_Type', 'Details', 'Create_Date',
       'gateway', 'Jenis_Laporan', 'Nama_Nasabah', 'No_Rekening', 'Nominal',
       'status', 'TanggalClosed', 'tanggalTransaksi', 'Chanel', 'Fitur',
       'Nomor_Kartu', 'user_group', 'assgined_to', 'attachment_done', 'email',
       'full_name', 'no_telepon', 'approver_login', 'approver_name',
       'SLAResolution', 'submitter_login_id', 'submitter_user_group',
       'user_login_name', 'Jenis_Produk', 'Last_Modified_By', 'Merchant_ID',
       'Modified_Date', 'NOTAS', 'Produk', 'SLA_Status', 'TID',
       'tanggalAttachmentDone', 'Tgl_Assigned', 'Tgl_Eskalasi', 'AnalisaSkils',
       'Attachment_', 'Bank_BRI', 'Biaya_Admin', 'Suku_Bunga', 'Bunga',
       'Butuh_Attachment', 'Cicilan', 'Hasil_Kunjungan', 'Log_Name',
       'MMS_Ticket_Id', 'Mass_Ticket_Upload_Flag', 'Nama_Supervisor',
       'Nama_TL', 'Nama_Wakabag', 'Nasabah_Prioritas', 'Notify_By',
       'Organization', 'Output_Settlement', 'phone_

## To Compare two files

In [18]:
# Load the newly uploaded files for detailed comparison
processed_file_path_newest = r"D:\dataquality\bricare_20230101_20230101.csv"
expected_file_path_newest = r"D:\dataquality\bricare_20230101_20230101.csv"

# Read the files
processed_df_newest = pd.read_csv(processed_file_path_newest)
expected_df_newest = pd.read_csv(expected_file_path_newest)

# Check for exact match first
exact_match = processed_df_newest.equals(expected_df_newest)

# Initialize a dictionary to store detailed comparison results
comparison_details = {
    'shape_match': processed_df_newest.shape == expected_df_newest.shape,
    'columns_match': processed_df_newest.columns.equals(expected_df_newest.columns),
    'column_differences': {},
    'value_differences': {}
}

# Compare each aspect if exact match is not true
if not exact_match:
    # Check shape
    if not comparison_details['shape_match']:
        comparison_details['shape_details'] = {
            'processed_shape': processed_df_newest.shape,
            'expected_shape': expected_df_newest.shape
        }
    
    # Check columns
    if not comparison_details['columns_match']:
        comparison_details['column_details'] = {
            'processed_columns': processed_df_newest.columns.tolist(),
            'expected_columns': expected_df_newest.columns.tolist()
        }

    # Check column-by-column values
    for column in processed_df_newest.columns:
        if not processed_df_newest[column].equals(expected_df_newest[column]):
            comparison_details['column_differences'][column] = processed_df_newest[column].compare(expected_df_newest[column])

# Summarize value differences
if not comparison_details['columns_match']:
    value_differences = {}
    for column in processed_df_newest.columns:
        if not processed_df_newest[column].equals(expected_df_newest[column]):
            value_differences[column] = processed_df_newest[column].compare(expected_df_newest[column])
    comparison_details['value_differences'] = value_differences

comparison_details


{'shape_match': True,
 'columns_match': True,
 'column_differences': {},
 'value_differences': {}}

# To collect all Error logs in a path

directory_path is the error logs path as well as the location where the combined error log file will be saved


In [41]:
import os
import pandas as pd

def combine_error_logs(directory_path):
    combined_df = pd.DataFrame()

    for filename in os.listdir(directory_path):
        if filename.startswith("error"):
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Normalize the 'TICKET_ID' to ensure duplicates are identified
    combined_df['TICKET_ID'] = combined_df['TICKET_ID'].str.upper()

    combined_df = combined_df.drop_duplicates(subset='TICKET_ID')

    output_path = os.path.join(directory_path, 'error_logs.csv')
    combined_df.to_csv(output_path, index=False)
    print(f"Combined error logs saved to: {output_path}")
    return output_path, combined_df

directory_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\log"
output_path, combined_df = combine_error_logs(directory_path)


Combined error logs saved to: C:\Users\maste\Downloads\dataloader_v60.0.2\log\error_logs.csv


# To cleanse user data

In [49]:
import pandas as pd
from io import StringIO
import html


# Function to decode HTML entities in a DataFrame
def decode_html_entities(df):
    df_decoded = df.applymap(lambda x: html.unescape(x) if isinstance(x, str) else x)
    return df_decoded


file_path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\USER.txt" 

# Read the raw file content
with open(file_path, 'r') as file:
    raw_data = file.readlines()
    
# Split headers and data    
header = raw_data[0].replace("&#124;", "|").strip()
data = [line.replace("&#124;", "|").strip() for line in raw_data[1:]]

# Read the cleaned data into a pandas DataFrame
df_cleaned = pd.DataFrame([line.split('|') for line in data], columns=header.split('|'))

# Decode HTML entities
df_cleaned = decode_html_entities(df_cleaned)
df_cleaned

# Remove the quotes in Dataframe
def remove_quotes(df):
    df.columns = df.columns.str.replace('"', '')
    df = df.apply(lambda col: col.str.replace('"', '', regex=True))
    return df

df_cleaned = remove_quotes(df_cleaned)
df_cleaned
df_cleaned.to_csv("user_data.csv", index=False)

C:\Users\maste\AppData\Local\Temp\ipykernel_43760\4048578058.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_decoded = df.applymap(lambda x: html.unescape(x) if isinstance(x, str) else x)


# Working on the data

In [3]:
import pandas as pd

path = r"D:\Salesforce\archive\dataquality\gx\BRICARE_25042024.csv"

# Option 1: Check for quoted fields or use different delimiter
try:
    df = pd.read_csv(path, sep=',', on_bad_lines='skip') 
except pd.errors.ParserError as e:
    print(f"Error parsing file: {e}")


C:\Users\maste\AppData\Local\Temp\ipykernel_15156\3036651791.py:7: DtypeWarning: Columns (1,8,17,21,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=',', on_bad_lines='skip')


In [4]:
import pandas as pd
import numpy as np

# Define the column list
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"
]

path = r"D:\Salesforce\archive\dataquality\gx\BRICARE_25042024.csv"

# Initialize lists to store data and problematic lines
data = []
problematic_lines = []

def parse_line(line):
    parts = line.split(',')
    try:
        # Identifying Ticket_ID
        ticket_id_index = next(i for i, part in enumerate(parts) if part.startswith('TTB'))
        ticket_id = parts[ticket_id_index]
        
        # Identifying Call_Type_ID
        call_type_id_index = ticket_id_index + 1
        call_type_id = parts[call_type_id_index]
        
        # Identifying Create_Date (the part that looks like a datetime string)
        create_date_index = next(i for i, part in enumerate(parts) if '-' in part and ':' in part)
        create_date = parts[create_date_index]
        
        # Call_Type is everything between Call_Type_ID and Create_Date
        call_type = ' '.join(parts[call_type_id_index + 1:create_date_index])
        
        # The rest of the fields in order
        rest = parts[create_date_index + 1:]
        
        # Combine into a single list in the correct order
        structured_line = [ticket_id, call_type_id, call_type, create_date] + rest
        
        # If the length is correct, return it
        if len(structured_line) == len(column_list):
            return structured_line
        # If there are extra fields, handle them (for example, by merging or ignoring)
        elif len(structured_line) > len(column_list):
            return structured_line[:len(column_list)]
        else:
            return None
    except StopIteration:
        # If any part of the parsing fails, consider the line problematic
        return None

# Read the file line by line and parse it
with open(path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    i = 0
    while i < len(lines):
        parsed_line = parse_line(lines[i])
        if parsed_line:
            data.append(parsed_line)
            i += 1
        else:
            # Check if merging the next line solves the issue
            if i + 1 < len(lines):
                merged_line = lines[i].strip() + ' ' + lines[i + 1].strip()
                parsed_line = parse_line(merged_line)
                if parsed_line:
                    data.append(parsed_line)
                    i += 2
                else:
                    problematic_lines.append((i, lines[i]))
                    i += 1
            else:
                problematic_lines.append((i, lines[i]))
                i += 1

# Convert the collected data into a DataFrame
df = pd.DataFrame(data, columns=column_list)


df['Ticket_ID'] = df['Ticket_ID'].astype(str)
df = df[df['Ticket_ID'].str.match(r'TTB\d+')]

df['Call_Type_ID'] = df['Call_Type_ID'].astype(str)
df = df[df['Call_Type_ID'].str.match(r'^\d{4}$')]

df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce')
df = df.dropna(subset=['Create_Date'])

df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')

columns_to_convert = ['TanggalClosed', 'tanggalTransaksi', 'Create_Date']
for column in columns_to_convert:
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
    df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)


df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')


startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricaredatareal_{formatted_startdate}_{formatted_enddate}.csv"


df.to_csv(filename, index=False)

print(f"Data saved to {filename}")


problematic_df = pd.DataFrame(problematic_lines, columns=['Line_Number', 'Data'])


problematic_filename = f"bricaredatareal_problematic_{formatted_startdate}_{formatted_enddate}.csv"
problematic_df.to_csv(problematic_filename, index=False)

print(f"Problematic data saved to {problematic_filename}")


print(f"Number of rows in dataframe: {len(df)}")
print(f"Number of problematic lines: {len(problematic_df)}")


Data saved to bricaredatareal_20200101_20200201.csv
Problematic data saved to bricaredatareal_problematic_20200101_20200201.csv
Number of rows in dataframe: 463581
Number of problematic lines: 881


# Create masked Data for UAT


To Mask:

Nama_Nasabah
No_Rekening
full_name
no_telepon
approver_name = beda dengan Nama_Nasabah
user_login_name = beda 
Log_Name
Nama_Supervisor
Nama_TL
Nama_Wakabag
Remark

Kolom Detail:

Kode Cabang          : 0307 4 digits
No Kartu             : 5221843130736932 16 digits
No Rekening          : 030701098507501 15 digits
Nama                 : SITI SHOLEHA
No ID                : 3316022012770004

In [13]:
import pandas as pd
from faker import Faker
import random 

fake = Faker('id_ID')
pd.set_option('display.max_columns', None)


path = r"D:\dataquality2\bricare_uat_20230101_20230101.csv"
df = pd.read_csv(path)

def mask_data(df):
    df['Nama_Nasabah'] = df['Nama_Nasabah'].apply(lambda x: fake.name())
    df['No_Rekening'] = df['No_Rekening'].apply(lambda x: fake.bban())
    df['full_name'] = df['full_name'].apply(lambda x: fake.name())
    df['no_telepon'] = df['no_telepon'].apply(lambda x: fake.phone_number())
    df['approver_name'] = df['approver_name'].apply(lambda x: fake.name())
    df['user_login_name'] = df['user_login_name'].apply(lambda x: fake.user_name())
    df['Log_Name'] = df['Log_Name'].apply(lambda x: fake.user_name())
    df['Nama_Supervisor'] = df['Nama_Supervisor'].apply(lambda x: fake.name())
    df['Nama_TL'] = df['Nama_TL'].apply(lambda x: fake.name())
    df['Nama_Wakabag'] = df['Nama_Wakabag'].apply(lambda x: fake.name())
    return df


df_masked = mask_data(df)

def generate_nik():
    return f'{fake.random_number(digits=16)}'

def mask_detail(detail):
    if isinstance(detail, float) and pd.isna(detail):
        return detail
    lines = str(detail).split('\n')
    masked_lines = []
    for line in lines:
        if 'Kode Cabang' in line:
            line = f'Kode Cabang          : {fake.random_number(digits=4)}'
        elif 'No Kartu' in line:
            line = f'No Kartu             : {fake.credit_card_number()}'
        elif 'No Rekening' in line:
            line = f'No Rekening          : {fake.bban()}'
        elif 'Nama' in line:
            line = f'Nama                 : {fake.name()}'
        elif 'No ID' in line:
            line = f'No ID                : {generate_nik()}'
        masked_lines.append(line)
    return '\n'.join(masked_lines)

# df_masked = df_masked.iloc[:10]


# Ensure 'Details' column is treated as string and apply mask_detail function
df_masked['Details'] = df_masked['Details'].astype(str).apply(mask_detail)



# gateway_options = [
#     'Email', 'Phone', 'Instagram', 'Walk-In', 'MMS', 
#     'Twitter', 'Facebook', 'BRImo', 'BRILink', 'Sabrina', 'Ceria'
# ]

#UAT
gateway_options = [
    'Email', 'Phone', 'Web', 'Facebook', 'Twitter'
]
# Assign random values to the 'gateway' column
df_masked['gateway'] = df_masked['gateway'].apply(lambda x: random.choice(gateway_options))




# df_masked = df_masked.iloc[:10]
df_masked
output_path = r"D:\dataquality2\bricare_uat_20230101_20230101_masking.csv"
df_masked.to_csv(output_path, index=False)


In [19]:
import pandas as pd

path=r"bricare_uat_20230101_20230101_masking.csv"
df=pd.read_csv(path)
df['status']='New'
df.to_csv(r"bricare_uat_20230101_20230101_masking_new.csv", index=False)

# To create data to test in SIT


In [94]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\SIT\dummy_data_case1100000.csv"

df=pd.read_csv(path) 
df=df.iloc[:60000]
# df['User*']='00GMR0000000dCf2AI'
df['User*']='00GMR0000000dEH2AY'
df['Status']='Escalated'
df['Status'] = df['Status'].replace('New', 'Escalated')
df['Case Origin']='Email'
df['Case Type']='a1BMR00000010h72AA'
df['Account']='001MR000004C7l2YAC'

df.to_csv('dummy_data_uat60k.csv', index=False)

# df['Merchant ID']
# df['TID']
# df['User*']

# To create data to test in UAT

In [92]:
import pandas as pd


path=r"D:\dataquality\bricare_20230101_20230101.csv"

df=pd.read_csv(path) 
df=df.iloc[:60000]
df.to_csv('dummy_data_case_uat60k.csv', index=False)

In [10]:
import pandas as pd


file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\CHM\CHM\Part2\bricare_get_cleansing_77_kolom(1).csv"
df = pd.read_csv(file_path, delimiter=';')


output_file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\bricare_get_cleansing_77_kolom_fixed_return.csv"
df=df.iloc[:1]
df['status']='Return'
df.to_csv(output_file_path, index=False)

# output_file_path

# Feature ID

In [54]:
import pandas as pd

# Load the CSV files
ref_fiturid_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\ref_fiturid.csv"
id_calltype_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_calltype_uat.csv"

ref_fiturid = pd.read_csv(ref_fiturid_path)
id_calltype = pd.read_csv(id_calltype_path)

# Convert calltype and NAME columns to string type
ref_fiturid['calltype'] = ref_fiturid['calltype'].astype(str)
id_calltype['NAME'] = id_calltype['NAME'].astype(str)

# Merge the dataframes to add the Call_Type_ID column based on the calltype column
ref_fiturid = ref_fiturid.merge(id_calltype[['NAME', 'ID']], left_on='calltype', right_on='NAME', how='left')

# Rename the ID column to Call_Type_ID
ref_fiturid.rename(columns={'ID': 'Call_Type_ID'}, inplace=True)

# Drop the unnecessary NAME column from the merge
ref_fiturid.drop(columns=['NAME'], inplace=True)

# Save the updated dataframe to a new CSV file
output_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\updated_fiturid_uat.csv"
# ref_fiturid=ref_fiturid.iloc[62:63]
ref_fiturid.to_csv(output_path, index=False)


print(f"Updated CSV file saved to {output_path}")



Updated CSV file saved to C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\updated_fiturid_uat.csv


# Knowledge

In [1]:
import pandas as pd

path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\knowledge\BRI - Detail BRICare(WI).csv"
df=pd.read_csv(path, encoding='ISO-8859-1')
df = df.fillna('')
df['Article Type']='Working Instruction'

output=r"C:\Users\maste\Downloads\dataloader_v60.0.2\knowledge\artikel_all.csv"



# Change 'Brilink' to 'BRILink' in 'Type' column
df['Type'] = df['Type'].replace('Brilink', 'BRILink')

# Remove rows with 'Wholesale' in 'Type' column
df = df[df['Type'] != 'Wholesale']

# df['Type'].unique()

df['URL_name'] = df['Call Type'].astype(str) + '-' + df['Call Type Info'].astype(str)

import re

# Define a function to clean the URL name
def clean_url_name(url_name):
    # Remove leading and trailing hyphens
    url_name = url_name.strip('-')
    # Replace invalid characters with hyphens and remove multiple hyphens
    url_name = re.sub(r'[^a-zA-Z0-9\u00C0-\u017F-]', '-', url_name)
    url_name = re.sub(r'-+', '-', url_name)
    # Remove leading and trailing hyphens again after replacement
    url_name = url_name.strip('-')
    return url_name

# Apply the cleaning function to the URL_name column
df['URL_name'] = df['URL_name'].apply(clean_url_name)



df.to_csv('artikel2.csv', index= False)
# df.iloc[:1].to_csv(output, index= False)

df



,Unnamed: 0,Call Type,Call Type Info,Type,User,Title,Detail Bricare,Deskripsi,Empati,Konfirmasi,...,Verifikasi,Gali Informasi,Pembuatan Laporan,Konfirmasi Ulang,Percepatan Komplain,"Solusi, Informasi",Edukasi & Cross Selling,Closing,Article Type,URL_name
0,1,1000,Nasabah Menanyakan Informasi Pengajuan Terkait...,CERIA,Agent Contact Center,1000 - Nasabah Menanyakan Informasi Pengajuan ...,nomor ponsel nasabah yang bisa dihubungi\nID C...,Nasabah menghubungi Contact BRI / datang ke Un...,Empati sesuai kondisi (pilih salah satu)\t\t\t...,Jika Nasabah menanyakan Cara & Syarat Pengajua...,...,1\tCara & Syarat pengajuan Aplikasi\nVerifikas...,1\tCara & Syarat pengajuan Aplikasi\nGali Info...,1\tCara & Syarat pengajuan Aplikasi\nBuat lapo...,,,1\tCara & Syarat pengajuan Aplikasi\nInformasi...,,Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1000-Nasabah-Menanyakan-Informasi-Pengajuan-Te...
1,2,1002,Nasabah Menyakan Terkait Promo dan Program CERIA,CERIA,Agent Contact Center,1002 - Nasabah Menyakan Terkait Promo dan Prog...,,Nasabah menghubungi Contact BRI / datang ke Un...,Empati sesuai kondisi (pilih salah satu)\nJika...,Konfirmasi :\nJika Nasabah menanyakan promo ce...,...,Verifikasi\nTidak dilakukan Verifikasi\n,Gali Informasi\t\n\t i. Jenis promo/...,Pembuatan Laporan\t\n\ti. Nomor laporan ti...,,,Informasi & Solusi \t\t\n\t i. \...,,Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1002-Nasabah-Menyakan-Terkait-Promo-dan-Progra...
2,3,1003,Nasabah Mengajukan Pelunasan Awal Cicilan CERIA,CERIA,Agent Contact Center,1003 - Nasabah Mengajukan Pelunasan Awal Cicil...,,Nasabah menghubungi Contact BRI / datang ke Un...,"Empati sesuai kondisi (pilih salah satu)\n""Bap...","Konfirmasi :\n""Untuk Permintaan Pelunasan awal...",...,Verifikasi ( Buka CLOS/ Finnachel )\t\nUntuk v...,Gali Informasi\t\n1.\tTanyakan alasan pelunasa...,Pembuatan Laporan \t\n1.\tBuat Laporan dengan ...,,,Informasi & Solusi\n1.\tCicilan telah berjalan...,,Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1003-Nasabah-Mengajukan-Pelunasan-Awal-Cicilan...
3,4,1005,Nasabah Mengajukan Pemblokiran CERIA,CERIA,Agent Contact Center,1005 - Nasabah Mengajukan Pemblokiran CERIA,Nasabah mengajukan pemblokiran Sementara Akun ...,Nasabah menghubungi Contact BRI / datang ke Un...,Empati\nJika nasabah infokan ingin melakukan p...,"Konfirmasi: \n""Untuk pemblokiran bersifat seme...",...,Verifikasi ( Buka WBS/ CLOS )\t\nUntuk verifik...,Gali Informasi\t\n1.\tTanyakan alasan perminta...,Pembuatan Laporan\nNasabah mengajukan pembloki...,,,Informasi & Solusi \t\n1.\tAgent Konfirmasi ke...,"Kalimat Edukasi\n""Kami informasikan Bank BRI t...",Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1005-Nasabah-Mengajukan-Pemblokiran-CERIA
4,5,1008,Nasabah Mengajukan Pengaktifan Akun Ceria Terb...,CERIA,Agent Contact Center,1008 - Nasabah Mengajukan Pengaktifan Akun Cer...,,,,,...,,,,,,,,,Working Instruction,1008-Nasabah-Mengajukan-Pengaktifan-Akun-Ceria...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,,,,,,,,,,,...,,,,,,,,,Working Instruction,
438,,,,,,,,,,,...,,,,,,,,,Working Instruction,
439,,,,,,,,,,,...,,,,,,,,,Working Instruction,
440,,,,,,,,,,,...,,,,,,,,,Working Instruction,


In [91]:
import pandas as pd

pd.set_option('display.max_columns', None)
df= pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error060424041601915.csv")
df.to_csv('error.csv', index=False)

In [23]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\knowledge\BRI - Detail BRICare 2.xlsx"

df = pd.read_excel(path)
df=df.iloc[:1]
# df['Gali Informasi']

df['URL_name'] = df['Call Type'].astype(str) + '-' + df['Call Type Info'].astype(str)

import re

# Define a function to clean the URL name
def clean_url_name(url_name):
    # Remove leading and trailing hyphens
    url_name = url_name.strip('-')
    # Replace invalid characters with hyphens and remove multiple hyphens
    url_name = re.sub(r'[^a-zA-Z0-9\u00C0-\u017F-]', '-', url_name)
    url_name = re.sub(r'-+', '-', url_name)
    # Remove leading and trailing hyphens again after replacement
    url_name = url_name.strip('-')
    return url_name

# Apply the cleaning function to the URL_name column
df['URL_name'] = df['URL_name'].apply(clean_url_name)

df.to_csv('artikel_bullet.csv', index=False)

# Generate Roles


## generate 1k roles

In [4]:
import pandas as pd

# Load the uploaded CSV file
file_path = '/mnt/data/extract_role.csv'
roles_df = pd.read_csv(file_path)

# Display the content of the CSV file to understand its structure
roles_df.head()


# Number of additional roles needed to reach a total of 1000
num_additional_roles_needed = 1000 - len(expanded_roles_df)

# Duplicate the existing roles to meet the required number of roles
additional_roles_more = expanded_roles_df.sample(num_additional_roles_needed, replace=True).reset_index(drop=True)

# Generate unique names for the additional roles
additional_roles_more['NAME'] = additional_roles_more['NAME'] + "_more_" + (additional_roles_more.index + 1).astype(str)

# Combine the original roles with the additional roles
expanded_roles_df_more = pd.concat([expanded_roles_df, additional_roles_more], ignore_index=True)

# Ensure we have a total of 1000 roles
expanded_roles_df_more = expanded_roles_df_more.head(1000)

# Save the expanded roles to a new CSV file
output_file_path_more = '/mnt/data/expanded_roles_1000.csv'
expanded_roles_df_more.to_csv(output_file_path_more, index=False)

output_file_path_more



FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/extract_role.csv'

## Check Extraction Data



### check data before 2023 = 27 columns

In [13]:
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"  
]

column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

import pandas as pd

path=r"D:\dataquality2\new as per 5 June\bricare_20221213_20240604_27_kolom.csv"
path2=r"D:\dataquality2\new as per 5 June\bricare_20221213_20240604_79_kolom.csv"
# df= pd.read_csv(path, delimiter=';')
df= pd.read_csv(path2, delimiter=';')


# column_list_set = set(column_list)
column_list_set = set(column_names)
df_columns_set = set(df.columns)

missing_columns = column_list_set - df_columns_set
extra_columns = df_columns_set - column_list_set

if not missing_columns and not extra_columns:
    print("The column names match.")
else:
    if missing_columns:
        print(f"Missing columns in DataFrame: {missing_columns}")
    if extra_columns:
        print(f"Extra columns in DataFrame: {extra_columns}")

################################################
# column_list_set2 = set(column_names)
# df_columns_set2 = set(df2.columns)

# missing_columns2 = column_list_set2 - df_columns_set2
# extra_columns2 = df_columns_set2 - column_list_set2

# if not missing_columns and not extra_columns:
#     print("The column names match.")
# else:
#     if missing_columns2:
#         print(f"Missing columns in DataFrame: {missing_columns2}")
#     if extra_columns2:
#         print(f"Extra columns in DataFrame: {extra_columns2}")


Missing columns in DataFrame: {'Tgl_Assigned'}
Extra columns in DataFrame: {'TglAssigned', 'cifno', 'Details'}


Data dengan 27 kolom:
- jumlah kolom sudah ok
- picklist Walk In harus diganti ke Walk-In
- attachment
- penambahan nama file diakhir "27"



Data dengan 79 kolom:
- picklist Walk In harus diganti ke Walk-In
- penambahan nama file diakhir "79"



Data Zendesk:
- harus hilangkan double quotes ""
- delimiternya ;
- nama file zendesk


Data Omni:
- SMS: harus format csv

In [2]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error060524083553095.csv"
df=pd.read_csv(path)
df['ERROR']

0    Unable to create/update fields: LastModifiedDa...
1    Unable to create/update fields: LastModifiedDa...
2    Unable to create/update fields: LastModifiedDa...
3    Unable to create/update fields: LastModifiedDa...
4    Unable to create/update fields: LastModifiedDa...
5    Unable to create/update fields: LastModifiedDa...
6    Unable to create/update fields: LastModifiedDa...
7    Unable to create/update fields: LastModifiedDa...
8    Unable to create/update fields: LastModifiedDa...
9    Unable to create/update fields: LastModifiedDa...
Name: ERROR, dtype: object

# Zendesk

In [33]:
import pandas as pd

file_path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\new as per 5 June\Data Zendesk 1-15 Januari 2024 - sampel.csv"

import csv


# Read the CSV file with proper handling of quotes and custom separator
with open(file_path, 'r', newline='') as file:
    reader = csv.reader(file, delimiter=',', quotechar='"')
    rows = [row for row in reader]

# Split the combined columns that have commas within them
split_rows = []
for row in rows:
    split_row = row[0].split(',') + row[1:]  # Split the first column and add the rest as they are
    split_rows.append(split_row)

# Create DataFrame from the processed rows
df = pd.DataFrame(split_rows[1:], columns=split_rows[0])

# Remove double quotes from column names
df.columns = df.columns.str.replace('"', '')

# Remove double quotes from all data in the DataFrame
df = df.replace('"', '', regex=True)

# Display the first few rows of the cleaned DataFrame
print(df.head())
df

# If you need to save the cleaned DataFrame to a new CSV file
# cleaned_file_path = '/mnt/data/Cleaned_Zendesk_Data.csv'
# df.to_csv(cleaned_file_path, index=False)

# print(f"Cleaned data saved to {cleaned_file_path}")



  Ticket ID Ticket channel   Assignee ID   Assignee name    Requester ID  \
0   3435272    Any channel  405257525413  Agent Sosmed 4  27064719563033   
1   3435273       Facebook  405257335633  Agent Sosmed 3  27064768687769   

     Requester name                                     Ticket subject  \
0        vianovia94  [IGC] Minimal opo boloo... ......#relsvideo #r...   
1  Masruroh Sodikin                                                  P   

  Requester created - Timestamp Ticket created - Timestamp  \
0           2024-01-01T00:03:30        2024-01-01T00:03:30   
1           2024-01-01T00:04:15        2024-01-01T00:04:15   

  Ticket solved - Timestamp                 Tickets  
0       2024-01-01T00:30:56  1.00000000000000000000  
1       2024-01-01T00:46:27  1.00000000000000000000  


,Ticket ID,Ticket channel,Assignee ID,Assignee name,Requester ID,Requester name,Ticket subject,Requester created - Timestamp,Ticket created - Timestamp,Ticket solved - Timestamp,Tickets
0,3435272,Any channel,405257525413,Agent Sosmed 4,27064719563033,vianovia94,[IGC] Minimal opo boloo... ......#relsvideo #r...,2024-01-01T00:03:30,2024-01-01T00:03:30,2024-01-01T00:30:56,1.00000000000000000000
1,3435273,Facebook,405257335633,Agent Sosmed 3,27064768687769,Masruroh Sodikin,P,2024-01-01T00:04:15,2024-01-01T00:04:15,2024-01-01T00:46:27,1.00000000000000000000


In [2]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\new as per 5 June\bricare_20221213_20240604_case_account.csv"
# df=pd.read_csv(path)
# df



path2=r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\new as per 5 June\case.csv"
df=pd.read_csv(path2)
df

ParserError: Error tokenizing data. C error: Expected 2 fields in line 86, saw 3


### Dummy data for Alex7

## Data Account Alex7

In [7]:
# Create dummy data for account with quoting (DONE)

import csv
import random
import os

def generate_dummy_data(file_path, file_name, num_rows):
    # Define possible values for each column
    account_names = ['John Doe', 'Jane Smith', 'Mike Brown', 'Lisa Green', 'Mark Taylor']
    account_owners = ['Owner A', 'Owner B', 'Owner C', 'Owner D']
    nasabah_types = ['Nasabah', 'Non Nasabah']
    account_record_types = ['Personal', 'Non Personal']

    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    # Writing to the CSV file
    with open(os.path.join(file_path, file_name), mode='w', newline='') as file:
        writer = csv.writer(file, quotechar='"', quoting=csv.QUOTE_ALL)  # Enforce quoting for all fields
        writer.writerow(['Account Name', 'CIF No', 'Account Owner', 'No Telp', 'Email', 'Nasabah Type', 'Account Record Type'])
        
        for _ in range(num_rows):
            account_name = random.choice(account_names)
            email = f"{account_name.split(' ')[0].lower()}.{account_name.split(' ')[1].lower()}@example.com"
            writer.writerow([
                account_name,
                ''.join(["{}".format(random.randint(0, 9)) for _ in range(10)]),  # 10-digit CIF No
                random.choice(account_owners),
                f'+62{random.randint(1000000000, 9999999999)}',  # Phone number
                email,
                random.choice(nasabah_types),
                random.choice(account_record_types)
            ])


file_path = 'D:\dataquality2'  # Adjust the path as needed
file_name = 'dummy_data_account.csv'
num_rows = 1  # Adjust the number of rows as needed

generate_dummy_data(file_path, file_name, num_rows)


## Data Case ALex 7


num of rows 

In [50]:
def create_salesforce_case_dummy_data(num_rows=100):
    import pandas as pd
    import random

    # Define the columns and possible values
    statuses = ["New", "Working", "Escalated"]
    types = ["Electronic", "Electrical", "Mechanical"]
    case_reasons = ["Performance", "Breakdown"]

    # Generate sample data
    data = {
        "Status": [random.choice(statuses) for _ in range(num_rows)],
        "Type": [random.choice(types) for _ in range(num_rows)],
        "Case Reason": [random.choice(case_reasons) for _ in range(num_rows)],
        "Subject": [f"Subject {i+1}" for i in range(num_rows)],
        "Description": [f"Description of issue {i+1}" for i in range(num_rows)],
        "Legacy_Ticket_ID": [f"TTB{i+1:06d}" for i in range(num_rows)],  # Generating 6-digit random numbers in order
    }

    # Create DataFrame
    df = pd.DataFrame(data)

    # Save to CSV
    file_path = 'dummy_casefor_alex5.csv'
    df.to_csv(file_path, index=False)

    return file_path

# Example usage
file_path = create_salesforce_case_dummy_data(num_rows=5)
file_path


'dummy_casefor_alex5.csv'

In [4]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\bricaredatareal_20200101_20200201.csv"

df=pd.read_csv(path)
df=df.iloc[:100]
df.to_csv('bricare_100_pakpondah.csv',index=False)

# data dummy



C:\Users\maste\AppData\Local\Temp\ipykernel_5108\2256201397.py:5: DtypeWarning: Columns (14,16,21,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path)


In [36]:
#To create dummy data to maintain the case number

import pandas as pd

def generate_dummy_file(filename, num_rows):
  
    data = {
        'Status': ['Cancelled'] * num_rows,
        'Priority': ['Low'] * num_rows,
        'Call Type' : ['1000'] * num_rows
    }
    
  
    df = pd.DataFrame(data)
    
   
    df.to_csv(filename, index=False)
    print(f"{filename} with {num_rows} rows created successfully.")


filename = 'dummy_data.csv'
num_rows = 150 
generate_dummy_file(filename, num_rows)


dummy_data.csv with 150 rows created successfully.


In [38]:
# The real data to be imported


import pandas as pd

# path=r"C:\Users\maste\Downloads\Test Data Dummy.csv"
# path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_150_casenumberandID.csv"
path=r"C:\Users\maste\Downloads\Test Data Dummy.csv"

df = pd.read_csv(path)


## Data Case ALex 7
Join the two files based on Ticket ID

In [46]:
# from the real data to join the ID or Case Number and update the records in Case 

import pandas as pd

# Load the data from the uploaded files
extract_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_150_casenumberandID.csv")
# test_data_dummy_df = pd.read_csv(r"C:\Users\maste\Downloads\Test Data Dummy.csv")
test_data_dummy_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error062124052713145.csv")


# Split the 'ID;"CASENUMBER"' column into two separate columns
extract_df[['ID', 'CASENUMBER']] = extract_df['ID;"CASENUMBER"'].str.split(';', expand=True)

# Drop the original combined column
extract_df.drop(columns=['ID;"CASENUMBER"'], inplace=True)

# Remove leading and trailing quotes from the columns
extract_df['ID'] = extract_df['ID'].str.strip('"')
extract_df['CASENUMBER'] = extract_df['CASENUMBER'].str.strip('"')

# Rename columns for easier merging
extract_df.rename(columns={'CASENUMBER': 'Ticket_ID'}, inplace=True)

# Merge the dataframes based on 'Ticket_ID'
merged_df = test_data_dummy_df.merge(extract_df, on='Ticket_ID', how='left')

# Reorder the columns to place 'ID' as the first column
columns = ['ID'] + [col for col in merged_df.columns if col != 'ID']
merged_df = merged_df[columns]

# Save the updated dataframe to a new CSV file
# merged_df=merged_df.iloc[:1]
merged_df['Call_Type_ID']='1000'
merged_df=merged_df.drop('ID', axis=1)
merged_df.to_csv('Updated_Test_Data_Dummy_2row.csv', index=False)


KeyError: 'Ticket_ID'

In [51]:
# the extracted data from Salesforce after being updated 


import pandas as pd

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\case_150.csv"

df=pd.read_csv(path, delimiter=';')
df2 = df[['ID', 'STATUS','CASENUMBER', 'SCC_LEGACY_TICKET_ID__C']]

# df3 = df2[df2['SCC_LEGACY_TICKET_ID__C'].notna()]
df2





,ID,STATUS,CASENUMBER,SCC_LEGACY_TICKET_ID__C
0,500MR000004NtSvYAK,Closed,TICKET000988612,TICKET000988612
1,500MR000004NtSwYAK,Closed,TICKET000988613,TICKET000988613
2,500MR000004NtSxYAK,Closed,TICKET000988614,TICKET000988614
3,500MR000004NtSyYAK,Cancelled,TICKET000988615,NaN
4,500MR000004NtSzYAK,Cancelled,TICKET000988616,NaN
5,500MR000004NtT0YAK,Cancelled,TICKET000988617,NaN
6,500MR000004NtT1YAK,Cancelled,TICKET000988618,NaN
7,500MR000004NtT2YAK,Cancelled,TICKET000988619,NaN
8,500MR000004NtT3YAK,Cancelled,TICKET000988620,NaN
9,500MR000004NtT4YAK,Closed,TICKET000988621,TICKET000988621


## Update the Supported Objects in Queue Name



In [35]:
import pandas as pd

# First Extract all the Queue ID from Group Object, you may use this query:
# SELECT Id
# where Type ='Queue'


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_group_queueid.csv"

df = pd.read_csv(path)

# add one column 'SObjectType' = Case, see the example below:
# QueueId,SObjectType
# 00GMR0000000trT2AQ,Case

df['SObjectType']='Case'
df.to_csv('extract_group_queueid.csv', index=False)

# Go to Data Loader 'insert' and choose  'QueueSobject' and insert the file with two columns: Id and SObjectType = Case

In [56]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error062124041536955.csv"


df = pd.read_csv(path)
df= df.drop('ERROR', axis=1)
df= df.drop('ERROR.1', axis=1)
df.to_csv(path, index=False)

KeyError: "['ERROR'] not found in axis"

In [1]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\sample_case (2)\bricare_20200806_20240430_0_27.csv"
df = pd.read_csv(path, delimiter=';')
df

,cifno,Ticket_ID,Call_Type_ID,Call_Type,Create_Date,gateway,Jenis_Laporan,Nama_Nasabah,No_Rekening,Nominal,...,attachment_done,email,full_name,no_telepon,approver_login,approver_name,SLAResolution,submitter_login_id,submitter_user_group,user_login_name
0,LD86852,TTB000002901285,8412.0,Nasabah BRI Gagal Transaksi Belanja di EDC BRI...,2020-08-06 16:52:14,Phone,Complaint - Transaction,b2714defcfef8c65d3154367beb604aa,40dc6606c2b863ebe08fcb22840cc9b8,23300800.0,...,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e
1,RCAW297,TTB000001778408,8812.0,Nasabah BRI gagal tarik tunai & terdebet di AT...,2020-08-06 16:52:14,Phone,Complaint - Transaction,b69e3868bc8b6c3231df7d691a5595fe,9f608bd9950ab2da8b30f77130f0685f,1000000.0,...,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,10.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e
2,AOY4038,TTB000001776805,8412.0,Nasabah BRI Gagal Transaksi Belanja di EDC BRI...,2020-08-06 16:52:14,Phone,Complaint - Transaction,948421d448fb51f1cf27584b1256a120,c8ba7ab42dd55eb9f6b04c7c0215c06d,246300.0,...,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e
3,AUA0263,TTB000002020684,8437.0,Nasabah BRI gagal transfer & terdebet jaringan...,2020-08-06 16:52:14,Phone,Complaint - Transaction,a4aa532c65d1503dcc41ef396d3c42b8,9ea2aedad78a4ceb57f234085797d272,100000.0,...,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e
4,HOL2556,TTB000025168832,8405.0,Kartu ATM BRI Tertelan di MESIN ATM,2020-08-06 16:52:14,Phone,Request,95d4f5421b5b387726b5d619dbe24155,8a69534ee0c414338ccdb16d5d485d57,0.0,...,NaN,NaN,fcc2dc136c7c3d69fb080cdd2b1648f1,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,90119221.0,LCC-CCTCALL,fcc2dc136c7c3d69fb080cdd2b1648f1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,RBU6378,TTB000025135298,8808.0,Pembayaran PLN Prabayar Gagal namun Saldo Terd...,2020-08-06 12:11:29,Phone,Complaint - Transaction,69c21efc4229135b07ec284b5fe00b75,70a10671ae2665c0341ea6edeebc9829,200000.0,...,NaN,NaN,a1e6cd3b223c1edfd8b14e1078172783,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,90105230.0,LCC-CCTCALL,a1e6cd3b223c1edfd8b14e1078172783
99996,MVM7576,TTB000002448185,8439.0,Nasabah BRI gagal transfer & terdebet jaringan...,2020-08-06 16:52:14,Phone,Complaint - Transaction,fff60542f6bed23d51f2cd25532002ca,4377f81f335e83f9b29f8cbc46b573a8,2000023.0,...,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e
99997,EY38563,TTB000002219828,8410.0,Kegagalan TAC / Transfer Lainnya Lewat BRI,2020-08-06 16:52:14,Phone,Complaint - Transaction,c63b1ad897ef8e5534c9ad341086d302,c54de8529029870946d1ddc8d818f6fd,4000000.0,...,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,10.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e
99998,RJN5271,TTB000025223346,8701.0,Blokir Kartu ATM karena kartu hilang,2020-08-06 16:52:14,Phone,Request,d66c9a37d803cdac680b06ef894fbe98,a87adddeab253a1f8bcbbc5616c264db,0.0,...,NaN,NaN,5067f8899729a5bf019cbaff71c5c882,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,60395.0,LCC-CCTCALL,5067f8899729a5bf019cbaff71c5c882
